In [1]:
import os
import sys
import open3d as o3d

sys.path.append("C:/Users/lucas/Desktop/UPC/MIT/pyfbx_i42-master/")

In [2]:
def fbx_to_py(fn):
    import pyfbx.parse_bin
    fbx_root_elem, fbx_version = pyfbx.parse_bin.parse(fn, use_namedtuple=True)
    py_data = fbx_root_elem, fbx_version
    return py_data


def py_to_fbx(fn, py_data):
    import pyfbx.encode_bin
    from pyfbx.encode_bin import FBXElem
    from pyfbx import data_types

    fbx_root_elem_in, fbx_version_in = py_data

    # re-build data
    def elem_conv_recursive(ele_in, ele_out):
        for i, data in enumerate(ele_in.props):
            data_type = ele_in.props_type[i]

            if data_type == data_types.BOOL:
                ele_out.add_bool(data)
            elif data_type == data_types.INT16:
                ele_out.add_int16(data)
            elif data_type == data_types.INT32:
                ele_outt.add_int32(data)
            elif data_type == data_types.INT64:
                ele_out.add_int64(data)
            elif data_type == data_types.FLOAT32:
                ele_out.add_float32(data)
            elif data_type == data_types.FLOAT64:
                ele_out.add_float64(data)
            elif data_type == data_types.BYTES:
                ele_out.add_bytes(data)
            elif data_type == data_types.STRING:
                ele_out.add_string(data)
            elif data_type == data_types.INT32_ARRAY:
                ele_out.add_int32_array(data)
            elif data_type == data_types.INT64_ARRAY:
                ele_out.add_int64_array(data)
            elif data_type == data_types.FLOAT32_ARRAY:
                ele_out.add_float32_array(data)
            elif data_type == data_types.FLOAT64_ARRAY:
                ele_out.add_float64_array(data)
            elif data_type == data_types.BOOL_ARRAY:
                ele_out.add_bool_array(data)
            elif data_type == data_types.BYTE_ARRAY:
                ele_out.add_byte_array(data)
            else:
                raise Exception("Unknown type %r" % data_type)

        for ele_iter_in in ele_in.elems:
            ele_iter_out = FBXElem(ele_iter_in.id)
            elem_conv_recursive(ele_iter_in, ele_iter_out)
            ele_out.elems.append(ele_iter_out)

    fbx_root_elem_out = FBXElem(b'')
    elem_conv_recursive(fbx_root_elem_in, fbx_root_elem_out)

    pyfbx.encode_bin.write(fn, fbx_root_elem_out, fbx_version_in)

In [3]:
fn_in = "C:/Users/lucas/Downloads/Borrar/fbx.fbx"
fn_in = "C:/Users/lucas/Desktop/UPC/MIT/tactile2object/intrinsics_extrinsics/data/20210719_2/recording_2021-07-19_17-01-51.fbx"
#fn_in = "C:/Users/lucas/Desktop/UPC/MIT/tactile2object/intrinsics_extrinsics/data/20210722/recording_2021-07-22_13-25-10.fbx"

py_data = fbx_to_py(fn_in)
a, num = py_data
objects = a.elems[8].elems

In [4]:
a.elems[2]

FBXElem(id=b'CreationTime', props=[b'2021-07-19 17:01:51:482'], props_type=bytearray(b'S'), elems=[])

In [5]:
for i in range(11):
    print(i, a.elems[i].id)

0 b'FBXHeaderExtension'
1 b'FileId'
2 b'CreationTime'
3 b'Creator'
4 b'GlobalSettings'
5 b'Documents'
6 b'References'
7 b'Definitions'
8 b'Objects'
9 b'Connections'
10 b'Takes'


In [6]:
i = 28
name = objects[i].props[1].decode("utf-8").split('\x00')[0]
translation = objects[i].elems[1].elems[2].props[-3:]
rotation = objects[i].elems[1].elems[3].props[-3:]

rotation 

[81.67082977294922, 14.704727172851562, -15.851095199584961]

In [51]:
import numpy as np
import math as m
  
def euler_to_rotMat(yaw, pitch, roll):
    Rz_yaw = np.array([
        [np.cos(yaw), -np.sin(yaw), 0],
        [np.sin(yaw),  np.cos(yaw), 0],
        [          0,            0, 1]])
    Ry_pitch = np.array([
        [ np.cos(pitch), 0, np.sin(pitch)],
        [             0, 1,             0],
        [-np.sin(pitch), 0, np.cos(pitch)]])
    Rx_roll = np.array([
        [1,            0,             0],
        [0, np.cos(roll), -np.sin(roll)],
        [0, np.sin(roll),  np.cos(roll)]])
    # R = RzRyRx
    rotMat = np.dot(Rz_yaw, np.dot(Ry_pitch, Rx_roll))
    return rotMat


def get_rotated_v(rot, vec):
    yaw, pitch, roll = rot
    R = euler_to_rotMat(psi, theta, phi)
    
    rot_vec = R @ vec
    
    return np.squeeze(np.asarray(rot_vec))

def get_finger(index):
    
    finger_01 = np.array(objects[index].elems[1].elems[2].props[-3:])
    finger_02 = np.array(objects[index + 1].elems[1].elems[2].props[-3:])
    finger_03 = np.array(objects[index + 2].elems[1].elems[2].props[-3:])
    finger_04 = np.array(objects[index + 3].elems[1].elems[2].props[-3:])
    
    rotation_01 = np.radians(objects[index].elems[1].elems[3].props[-3:])
    rotation_02 = np.radians(objects[index + 1].elems[1].elems[3].props[-3:]) 
    rotation_03 = np.radians(objects[index + 2].elems[1].elems[3].props[-3:]) 
    rotation_04 = np.radians(objects[index + 3].elems[1].elems[3].props[-3:]) 

    finger_01 = get_rotated_v(rotation_01, finger_01)
    finger_02 = get_rotated_v(rotation_02, finger_02) + finger_01
    finger_03 = get_rotated_v(rotation_03, finger_03) + finger_02
    finger_04 = get_rotated_v(rotation_04, finger_04) + finger_03

    finger = [finger_01, finger_02, finger_03, finger_04]
    
    return finger

In [52]:

all_fingers = get_finger(28) + get_finger(32) + get_finger(36) + get_finger(40) + get_finger(44)

hand_xyz = np.vstack((all_fingers))

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(hand_xyz)

mesh = o3d.geometry.TriangleMesh.create_coordinate_frame()
mesh.scale(0.05, center=(0,0,0))

o3d.visualization.draw_geometries([pcd])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


In [53]:
index = 28

finger_01 = np.array(objects[index].elems[1].elems[2].props[-3:])
finger_02 = np.array(objects[index + 1].elems[1].elems[2].props[-3:])

rotation_01 = np.radians(objects[index].elems[1].elems[3].props[-3:])
rotation_02 = np.radians(objects[index + 1].elems[1].elems[3].props[-3:]) 


In [54]:
finger_01

array([ 0.03621711, -0.00050032, -0.03519914])

In [50]:
import numpy as np
import math

def rotation_matrix(axis, theta):
    """
    Return the rotation matrix associated with counterclockwise rotation about
    the given axis by theta radians.
    """
    axis = np.asarray(axis)
    axis = axis / math.sqrt(np.dot(axis, axis))
    a = math.cos(theta / 2.0)
    b, c, d = -axis * math.sin(theta / 2.0)
    aa, bb, cc, dd = a * a, b * b, c * c, d * d
    bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
    return np.array([[aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)],
                     [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
                     [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])

def get_rot(rot, v):
    psi, theta, phi = rot
    
    axis = [0, 1, 0]
    v = np.dot(rotation_matrix(axis, theta), v)
    
    axis = [1, 0, 0]
    v = np.dot(rotation_matrix(axis, psi), v)
    

    axis = [0, 0, 1]
    v = np.dot(rotation_matrix(axis, phi), v)
    
    return v

In [44]:
from scipy.spatial.transform import Rotation as Rotation

r = Rotation.from_rotvec(rotation_01)
R = r.as_matrix()


array([[ 0.94079912,  0.33883683,  0.00930652],
       [-0.03466902,  0.1234996 , -0.99173883],
       [-0.33718699,  0.93270437,  0.12793546]])